In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

## 1. Загрузка и обработка данных

In [7]:
data_of_iris = load_iris()
iris_df = pd.DataFrame(data=data_of_iris.data, columns=data_of_iris.feature_names)
iris_df['iris_class'] = data_of_iris.target

In [8]:
iris_df = iris_df[iris_df['iris_class'] != 0].reset_index().drop('index', axis=1)
iris_df['iris_class'] = np.where(iris_df['iris_class'] == 1, 0, 1)
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),iris_class
0,7.0,3.2,4.7,1.4,0
1,6.4,3.2,4.5,1.5,0
2,6.9,3.1,4.9,1.5,0
3,5.5,2.3,4.0,1.3,0
4,6.5,2.8,4.6,1.5,0


## 2. Разделение данных на X и y

In [9]:
X = iris_df.drop('iris_class', axis=1)
y = iris_df['iris_class']

## 3. Реализация логистической регрессии

In [10]:
def log_reg_model(X, weight):
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))
    
    X = np.array(X)
    predict_array = [sigmoid(X[i] @ weight) for i in range(len(X))]
    return np.array(predict_array)

def cross_entropy(y_true, y_pred):
    logloss_1 = np.sum(np.log(y_pred[y_true == 1] + 1e-30))
    logloss_0 = np.sum(np.log(1 - y_pred[y_true == 0] + 1e-30))
    logloss_total = -(logloss_0 + logloss_1) / len(y_true)
    return logloss_total

def gradient_fit(X, y):
    def gradient(X, weight, y_true):
        y_pred = log_reg_model(X, weight)
        gradient = np.matmul(X.T, y_pred - np.array(y_true))
        return np.array(gradient)

    np.random.seed(42)
    EPSILON = 0.001 # значение для выхода
    EPOCHS = 200 # кол-во итераций
    LEARNING_RATE = 0.0001 # шаг градиента

    WEIGHT = np.random.randn(X.shape[1]) # случайная начальная точка

    next_weight = WEIGHT

    for i in range(EPOCHS):
        current_weight = next_weight

        # движение в негативную сторону вычисляемого градиента
        next_weight = current_weight - LEARNING_RATE * gradient(X, next_weight, y)
    
        if np.linalg.norm(current_weight - next_weight) <= EPSILON:
            break

        if i % 20 == 0:
            print(f"Итерация: {i}")
            print(f'Текущие веса модели: {current_weight}')
            y_pred = log_reg_model(X, next_weight)
            y_pred_class = np.where(y_pred >= 0.5, 1, 0)
            accuracy = (y_pred_class == y).sum() / len(y)
            print(f"Logloss {cross_entropy(y, y_pred)}")
            print(f"Accuracy {accuracy}")
            print("--------------------------------------------------------")

def accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)

## 4. Метод градиентного спуска

In [14]:
gradient_fit(X, y)

Итерация: 0
Текущие веса модели: [ 0.49671415 -0.1382643   0.64768854  1.52302986]
Logloss 3.515850082146899
Accuracy 0.5
--------------------------------------------------------
Итерация: 20
Текущие веса модели: [-0.06731073 -0.40157152  0.24356275  1.39744487]
Logloss 0.7692549552528724
Accuracy 0.5
--------------------------------------------------------
Итерация: 40
Текущие веса модели: [-0.23058979 -0.4805085   0.14531428  1.37266362]
Logloss 0.5062335061605593
Accuracy 0.92
--------------------------------------------------------
Итерация: 60
Текущие веса модели: [-0.24819605 -0.49268611  0.16028301  1.3859071 ]
Logloss 0.5019104768292688
Accuracy 0.92
--------------------------------------------------------
Итерация: 80
Текущие веса модели: [-0.26162612 -0.50292385  0.17825885  1.40010337]
Logloss 0.4978659633967662
Accuracy 0.92
--------------------------------------------------------
Итерация: 100
Текущие веса модели: [-0.27483934 -0.51303245  0.19613199  1.41419518]
Logloss 0

> Как мы видим модель обучилась довольно хорошо, мы нашли оптимальные веса.